# Homework exercise 3
## Deadline: upload to Moodle by 7 June 18:00 h

__Please submit your homework either as a Jupyter Notebook or using .py files.__

If you use .py files, please also include a PDF containing the output of your code and your explanations. Either way, the code needs to be in a form that can be easily run on another computer.

__Name 1:__

__Name 2:__

__Name 3:__


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 26 May to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### APIs, NLP, ML


__Reddit__


The Reddit offers an API providing free access to up to 1000 submissions (i.e. posts), comments on those submissions, and some meta data. `PRAW` is a module that facilitates usage of the Reddit API, and it is recommended that you use it. It is documented at https://praw.readthedocs.io/en/latest/index.html and the Quick Start page should give you most of the information you need.

(If you would like to use more Reddit data, you may want to consider `PSAW`. You won't need it for this homework exercise, though it might be of interest if you'd like to use Reddit in your course project.)

Please note that using the Reddit API requires users to register. Feel free to use any temporary email address if you have any concerns with sharing your usual address with Reddit.

Please collect data for the top 1000 submissions and all the comments on those submissions for the subreddits 'wallstreetbets' and 'StockMarket'.

The goal will be to classify submissions, i.e., to predict which subreddit each submission belongs to. 

Write a class RedditClassifier that contains two object attributes: the categorical variable to be used in the classification (here: the subreddit) and a list (for now, since you have only encountered a binary classifier, of length 2) of values for that category (i.e. a list of the names of the two subreddits). The class should be derived from the Perceptron class from Chapter 7. It should contain a method that combines the preprocessing and the estimation.

The preprocessing should generate features that might help predict the category. Please don't use features such as the frequency of each word as we will take a look at appropriate ways of working with such information only in the next class. Possible features could be: length of title, number of comments, average sentence length, word length, number of sentences in the comments, shares of nouns, verbs, etc., number of stop words used compared to other words, etc.. Please add any other features that you think might be useful for this task but don't include features that are likely to perfectly predict the category (e.g. the author of a submission).

Please use this class to classify the submissions. Plot the percentage of submissions that are incorrectly classified in each epoch for different learning rates. Don't expect to achieve near-perfect predictions as the features you employ may not be able to easily distinguish between categories the way they did in the example in class.

In [189]:
import praw
from praw.models import MoreComments
import pandas as pd
import numpy as np

subreddits = ["StockMarket", 'wallstreetbets']
reddit = praw.Reddit(
    client_id="cuDmFCTelGCG4w",
    client_secret="JJKHv_s7LlxkTKJE90xkPMiZIEqFRQ",
    user_agent="Homework3"
)

wallstreetbets=reddit.subreddit('wallstreetbets')
stockmarket=reddit.subreddit('StockMarket')


hot_stockmarket=stockmarket.hot(limit=1000)

hot_wsb=wallstreetbets.hot(limit=1000)

values = [hot_stockmarket, hot_wsb]
labels = ["StockMarket", 'wallstreetbets']

# 



In [190]:
for subred in values:
    print(subred)

In [191]:
# for subred in values:
#     for submission in subred:
#         print(submission.id)

In [192]:
class Reddit_Perceptron(object):
    """Perceptron classifier.
    Parameters
    ------------
    eta : float
    Learning rate (between 0.0 and 1.0)
    n_iter : int
    Passes over the training dataset.
    random_state : int
    Random number generator seed for random weight
    initialization.
    Attributes
    -----------
    w_ : 1d-array
    Weights after fitting.
    errors_ : list
    Number of misclassifications (updates) in each epoch.
    """

    def __init__(self, X, y, eta=0.01, n_iter=50, random_state=1):
        self.X = X
        self.y = y
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    # def __iter__(self):
    #     return (subred for subred in self.X)
        

    def preprocessing(self):
        titles=[]
        comment_count=[]
        comments=[]
        over_18=[]
        upvote_ratio=[]
        num_comments=[]
        text=[]
        spoiler=[]
        lentitles=[]
        isself=[]
        subred=[]
        
        for subredit, label in zip(self.X, self.y):
            for submission in subredit:
                subred.append(label)
                # print(submission.id)
                over_18.append(submission.over_18)
                upvote_ratio.append(submission.upvote_ratio)
                comment_count.append(submission.num_comments)
                titles.append(submission.title)
                if submission.selftext=='': 
                    text.append(0)
                else: 
                    text.append(submission.selftext)
                spoiler.append(submission.spoiler)
                #Whether or not the submission is a selfpost (text-only).
                isself.append(submission.is_self)

            for title in titles:
                lentitles.append(len(title))


        # for column in [titles, comment_count, comments,over_18,upvote_ratio,num_comments,text,spoiler,lentitles,isself]:
        #     print(len(column))

        data = pd.DataFrame(
        {#'Titles': titles,
         'Comment Count':comment_count,
         #'Comments':comments,
         'Over 18':over_18, 
         'Upvotoe Ratio':upvote_ratio,
         #'Num Comments':num_comments, 
         #'Text':text, 
         'Spoiler':spoiler, 
         'Isself':isself
        })
        return data.values, subred


    def fit(self):
        """Fit training data.
        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
        Training vectors, where n_samples is the number of samples and
        n_features is the number of features.
        y : array-like, shape = [n_samples]
        Target values.
        Returns
        -------
        self : object
        """
        X, y = self.preprocessing()
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
        for xi, target in zip(X, y):
            update = self.eta * (target - self.predict(xi))
            self.w_[1:] += update * xi
            self.w_[0] += update
            errors += int(update != 0.0)
        self.errors_.append(errors)
        return self
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    def predict(self,X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)


In [193]:
red_perceptron = Reddit_Perceptron(X = values, y= labels)
red_perceptron.fit()


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U11'), dtype('<U11')) -> dtype('<U11')

In [183]:
print(red_perceptron.errors)

671
671


In [182]:
data

array([[329, False, 0.95, False, False],
       [60, False, 0.94, False, False],
       [19, False, 0.93, False, False],
       ...,
       [27, False, 0.89, False, False],
       [63, False, 0.82, False, True],
       [61, False, 0.96, False, False]], dtype=object)